In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from Load import json_to_df
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer

Using TensorFlow backend.


In [ ]:
df_music = json_to_df("data\\music_reviews_train.json")

In [ ]:
df_music_dev = json_to_df("data\\music_reviews_dev.json")

In [ ]:
df_music=df_music.replace({"sentiment":{"positive":1,"negative":0}})
df_music_dev=df_music_dev.replace({"sentiment":{"positive":1,"negative":0}})

In [ ]:
df_music = df_music.replace(np.nan, '', regex=True)
df_music['reviewText'] = df_music.reviewText + ' ' + df_music.summary

In [ ]:
df_music_dev = df_music_dev.replace(np.nan, '', regex=True)
df_music_dev['reviewText'] = df_music_dev.reviewText + ' ' + df_music_dev.summary

In [ ]:
train_text = df_music["reviewText"].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = df_music["sentiment"].tolist()

In [ ]:
test_text = df_music_dev["reviewText"].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = df_music_dev["sentiment"].tolist()

In [ ]:
#GPU_OPTIONS = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#CONFIG = tf.ConfigProto(gpu_options=GPU_OPTIONS)
#sess = tf.Session(config = CONFIG)

In [ ]:
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
class ElmoEmbeddingLayer(layers.Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
        
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [13]:
input_text = layers.Input(shape=(1,), dtype="string")
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_text], outputs=pred)

"""
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=5,
          batch_size=384)
"""

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


"\nmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nmodel.summary()\nmodel.fit(train_text, \n          train_label,\n          validation_data=(test_text, test_label),\n          epochs=5,\n          batch_size=384)\n"

In [14]:
model.load_weights("model_")

In [12]:
model = None

In [15]:
music = pd.read_csv("Experiment results\\hardcases_beauty_cheatsheet.csv").replace({"positive":1,"negative":0})

In [6]:
music

,Unnamed: 0,review_body,sentiment,casetype
0,0,I have now seen the future of country/bluegras...,1,fairness
1,1,Bought the album for my son. He has cerebal p...,1,fairness
2,2,I can't believe all the 4 and 5 stars this rec...,0,invariance
3,3,I love t.A.T.u. I'm a lesbian myself and to he...,1,fairness
4,4,I bought this record in order to do the class ...,0,fairness
...,...,...,...,...
95,95,"I used to like this music, although now I regr...",0,temporal
96,96,You will never here the Symphonie Fantastique ...,1,regular
97,97,"In the past I thought this cd was excellent, b...",0,temporal
98,98,I bought this CD because I'd heard so many gre...,0,regular


In [16]:
hard_text = [' '.join(t.split()) for t in music["review_body"]]
hard_text = np.array(hard_text, dtype=object)[:, np.newaxis]

In [17]:
results = model.predict(hard_text)

In [24]:
tf.greater(results, .5)

<tf.Tensor 'Greater_2:0' shape=(100, 1) dtype=bool>

In [18]:
preds = []
for i in results:
    if i > 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(preds, music["sentiment"])

0.69